In [1]:
from core.generate_streets import read_overture_region_streets

import gc
import glob

import geopandas as gpd
import momepy as mm
import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [3]:
streets_dir = '/data/uscuni-ulce/overture_streets/'
regions_datadir = "/data/uscuni-ulce/"

In [4]:
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )
region_hulls.shape

(828, 1)

In [16]:
# %%capture cap

# for region_id, region_hull in region_hulls.to_crs(epsg=4326).iterrows():\

#     if region_id != 69333: continue
#     print('Processing', region_id, datetime.datetime.now())
#     streets = download_overture_streets(region_hull.iloc[0], region_id)
#     streets.to_parquet(streets_dir + f'streets_{region_id}.pq')

In [17]:
def process_and_save(region_hull, region_id):
    print('Processing', region_id, datetime.datetime.now())
    streets = read_overture_region_streets(region_hull, region_id)
    streets.to_parquet(streets_dir + f'streets_{region_id}.pq')

In [18]:
downloaded_streets = glob.glob(streets_dir + '*')
downloaded_streets = [int(s.split('_')[-1][:-3]) for s in downloaded_streets]

In [19]:
region_hulls = region_hulls.loc[~region_hulls.index.isin(downloaded_streets), ]
region_hulls

,convex_hull
labels,


In [ ]:
%%capture cap

from joblib import Parallel, delayed

n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_and_save)(region_hull.iloc[0], region_id) for region_id, region_hull in region_hulls.to_crs(epsg=4326).iterrows()
)


In [29]:
gdf = gpd.read_parquet(streets_dir + f'streets_{69333}.pq')

In [32]:
# gdf.explore()

1

In [21]:
cap.stderr

'/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.\n  warnings.warn(\n'

In [22]:
gdf = gpd.read_parquet(streets_dir + f'streets_{69333}.pq')

In [ ]:
for region_id, region_hull in region_hulls.iterrows():
    streets.append(streets.to_parquet(streets_dir + f'streets_{region_id}.pq'))